## Figure 3: Geometry comparison between RGI6 and RGI7

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

cl = px.colors.qualitative.D3
os.chdir('/home/rooda/Dropbox/Patagonia')

## Data

In [ ]:
# data
RGI6   = gpd.read_file("GIS South/Glaciers/RGI6_v2.shp")
RGI7   = gpd.read_file("GIS South/Glaciers/RGI7_v2.shp")

RGI6["year"] = RGI6.BgnDate.str[0:4].astype("int16")
RGI7["year"] = RGI7.src_date.str[0:4].astype("int16")

# polygons to points (centroid)
basins = gpd.read_file("GIS South/Basins_Patagonia_ice.shp")
basins = basins.set_index("ID")

names = ["Yelcho", "Baker", "Santa Cruz                             ", "Palena", "Grey", "Puelo", "Cisnes", "Aysen", "Pascua"]
basins.loc[basins.basin_area > 5000, "Name"] = names

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)
geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords = [-43.2, -46.0,   -46.9,   -47.6,   -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords = [-71.2, -71.7,   -73.8,   -71.7,   -72.2,   -72.3,   -72.1, -72.8, -69.2]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names]

## Plot

In [ ]:
fig = make_subplots(rows=2, cols=3, horizontal_spacing = 0.05, vertical_spacing = 0.08, subplot_titles = ["a) Area difference(%)","b) Area (log km<sup>2</sup>)",
                                                                                                          "c) Slope (deg)","d) Aspect (deg)","e) Acquisition date (year)"],
                    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "xy"}, {"type": "xy"}],
                           [None,                                 {"type": "xy"}, {"type": "xy"}]])

# Map ----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["rgba(213,213,213,0.7)", "rgba(213,213,213,0.7)"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = ((basins['RGI7_area']/basins['RGI6_area'])-1)*100, 
                            colorscale = [cl[1], "#ffe9ba" ,cl[0]], marker_line_color ='rgba(255,255,255, 0.6)', marker_line_width=0.5,
                            zmin = -100, zmax = 100, colorbar=dict(len=0.5, x=0.23, y= 0.75, title='Area (%)', thickness=20)), row=1, col=1)

fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.7,color = 'black'),opacity = 0.5, showlegend = False),row=1, col=1)
fig.add_trace(go.Scattergeo(lon=lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=9, color = "rgba(0,0,0,0.7)")),row=1, col=1)

fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['Name'], mode = 'text', showlegend = False, 
                   textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=1)

fig.update_geos(showframe = True, framecolor = "rgba(0,0,0,0.5)", framewidth = 1, lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], bgcolor = "#f9f9f9", showland = False, showcoastlines = False, showlakes = False)

# Area in log km2 --------------------------------------------------------------------------------------------------
fig.add_trace(go.Histogram(x=np.log(RGI6.area_km2), histnorm='percent', marker_color= cl[1], name = "RGI6"), row=1, col=2)
fig.add_trace(go.Histogram(x=np.log(RGI7.area_km2), histnorm='percent', marker_color= cl[0], name = "RGI7"), row=1, col=2)
fig.update_yaxes(range = (0,4), dtick = 1, row=1, col=2)
fig.update_xaxes(tickmode = 'array', row=1, col=2)
fig.update_xaxes(tickvals = [-4, -2, 0, 2, 4, 6], ticktext = ["10<sup>-4<sup>", "10<sup>-2<sup>", "10<sup>0<sup>", "10<sup>2<sup>", "10<sup>4<sup>", "10<sup>6<sup>"], row=1, col=2)
fig.update_traces(opacity=0.7, row = 1, col = 2)

# Slope in deg --------------------------------------------------------------------------------------------------
fig.add_trace(go.Histogram(x=RGI6.slope_v2, histnorm='percent', marker_color= cl[1], showlegend = False, xbins=dict(start=0, size=1, end=60)), row=1, col=3)
fig.add_trace(go.Histogram(x=RGI7.slope_v2, histnorm='percent', marker_color= cl[0], showlegend = False, xbins=dict(start=0, size=1, end=60)), row=1, col=3)
fig.update_yaxes(range = (0,8), dtick = 2, row=1, col=3)
fig.update_xaxes(range = (0,60), dtick = 15, row=1, col=3)
fig.update_traces(opacity=0.7, row = 1, col = 3)

# Aspect in deg ----------------------------------------------------------------------------------------------------
fig.add_trace(go.Histogram(x=RGI6.aspect_v2, histnorm='percent', marker_color= cl[1], showlegend = False), row=2, col=2)
fig.add_trace(go.Histogram(x=RGI7.aspect_v2, histnorm='percent', marker_color= cl[0], showlegend = False), row=2, col=2)
fig.update_yaxes(dtick = 1, range = (0,5), row=2, col=2)
fig.update_traces(opacity=0.7, row = 2, col = 2)

# RGI year ---------------------------------------------------------------------------------------------------------
fig.add_trace(go.Histogram(x=RGI6.year, histnorm='percent', marker_color= cl[1], showlegend = False), row=2, col=3)
fig.add_trace(go.Histogram(x=RGI7.year, histnorm='percent', marker_color= cl[0], showlegend = False), row=2, col=3)
fig.update_xaxes(dtick = 3, row=2, col=3)
fig.update_yaxes(range = (0,100), row=2, col=3)
fig.update_traces(opacity=0.7, row = 2, col = 3)

# All -----------------------------------------------------------------------------------------------------
fig.update_xaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True)
fig.update_yaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", griddash = "dot", mirror=True, title = "Percent (%)", title_standoff = 0)

fig.update_layout(font=dict(size=14), barmode='overlay', plot_bgcolor="rgba(213,213,213,0.6)", margin = dict(l=5, r=5, b=5, t=30), autosize=False, width=1350, height=800)
fig.update_layout(legend=dict(y=0.99, x=0.93, bgcolor = 'rgba(255,255,255,0.5)'))
fig.show()

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_3_RGI.png", scale=4)

## Hexbins alternative of  for a)

In [ ]:
fig = ff.create_hexbin_mapbox(
    data_frame=RGI, lat="CenLat", lon="CenLon",
    nx_hexagon=15, opacity=0.7, labels={"color": "ΔGlaciers (n)"},
    min_count=1, color="number", agg_func=np.sum, color_continuous_scale=[(0, c2), (0.16, "white"), (1, c1)], 
    range_color = [-50,250], mapbox_style = "carto-positron")

fig.update_mapboxes(style='carto-positron', center={'lat': -48.5, 'lon': -72},  zoom=5.3)

fig.update_layout(font=dict(size=20), autosize=False, width=1000, height=1500)
fig.update_layout(margin = {'l':0.1,'r':0.1,'t':0.1,'b':0.1})
fig.show()